In [26]:
import pandas as pd
import requests
import time
import datetime
import urllib.parse
import csv
import os
import requests
import Funciones.tratarJsonYCSV as FncJsonCSV
from bs4 import BeautifulSoup
from modules.oracle import OracleDataInserter

In [27]:
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
BERA_CONEXION = "karstec"
BERA_ESQUEMA = "KARSTEC"

bera_con = OracleDataInserter(schema=BERA_ESQUEMA, glue_con=BERA_CONEXION)

In [28]:
listado = bera_con.ejecutar_consulta_sql(
            sql=f"""
                SELECT DISTINCT RESP_PAGO
                FROM KARSTEC_PROD.COPIA_RESP_PAGO
                WHERE ROWNUM <= 10
            """)

datos_karstec = listado
columna_resp_pago = ('RESP_PAGO',)

df_listado = pd.DataFrame(datos_karstec, columns=columna_resp_pago)

In [29]:
df_listado['PROCESADO'] = 'PROCESADO'

,RESP_PAGO,PROCESADO
0,AVILA MERCEDES DOMINGA,PROCESADO
1,GODOY SANDRA INES,PROCESADO
2,HINOJO ESTER,PROCESADO
3,FATTORE JUAN,PROCESADO
4,SPAVENTO RICARDO-PONCE ALBINA,PROCESADO
5,SOLIS EDUARDO DANIEL,PROCESADO
6,CHARKIEWICZ LUISA,PROCESADO
7,PASTORE ZULEMA GUILLERMINA,PROCESADO
8,SISMAEL ERICA ALEJANDRA,PROCESADO
9,GRAZIANO ROBERTO OSMAR,PROCESADO


In [24]:
def primer_busqueda_y_guardado(df_listado):
    resultados_cuit = []

    # Crear un DataFrame para almacenar los resultados
    df_resultados = pd.DataFrame(columns=['nombre', 'apellido', 'cuit'])

    # Dividir la lista en lotes de 10 elementos
    num_lotes = len(df_listado) // 10 + (1 if len(df_listado) % 10 > 0 else 0)
    print(f'Se armaron {num_lotes} lotes para la búsqueda inicial.')

    for i in range(0, len(df_listado), 10):
        lotes = df_listado.iloc[i:i+10]  # Obtener el lote actual
        nombres_apellidos = []  # Lista para almacenar nombres y apellidos del lote actual
        
        for index, row in lotes.iterrows():
            partes = row['RESP_PAGO'].split(" ")
            nombre = partes[0]
            apellido = " ".join(partes[1:])
            nombres_apellidos.append((nombre, apellido))

        for nombre, apellido in nombres_apellidos:
            url = 'https://www.cuitonline.com/search.php?q=' + apellido + '%20' + nombre
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            resultados = soup.find_all('div', class_='hit')

            if resultados:
                cuits = []
                for resultado in resultados:
                    cuit_elem = resultado.find('span', class_='cuit')
                    if cuit_elem:
                        cuit = cuit_elem.text.strip().replace('-', '')
                        cuits.append(cuit)
                time.sleep(30)

                if cuits:
                    for cuit in cuits:
                        resultados_cuit.append({"nombre": nombre, "apellido": apellido, "cuit": cuit})
                else:
                    resultados_cuit.append({"nombre": nombre, "apellido": apellido, "cuit": 'No encontrado'})

            else:
                resultados_cuit.append({"nombre": nombre, "apellido": apellido, "cuit": 'No encontrado'})

        print(f'Se va procesando lote {i//10 + 1} de {num_lotes} en la búsqueda inicial.')

    # Llenar el DataFrame de resultados
    df_resultados = pd.DataFrame(resultados_cuit)

    df_resultados.rename(columns=lambda x: x.upper(), inplace=True)

    if not df_resultados.empty:
                bera_con.insert_data(
                    df=df_resultados,
                    table_name="CUIT_ONLINE_PBUS",
                    mode="Append",
                )
    
    return df_resultados

In [25]:
resultados_cuit = primer_busqueda_y_guardado(df_listado)

Se armaron 1 lotes para la búsqueda inicial.
Se va procesando lote 1 de 1 en la búsqueda inicial.


In [23]:
resultados_cuit

,nombre,apellido,cuit
0,AVILA,MERCEDES DOMINGA,No encontrado
1,GODOY,SANDRA INES,No encontrado
2,HINOJO,ESTER,27256505792
3,HINOJO,ESTER,23057475404
4,FATTORE,JUAN,20228844676
5,FATTORE,JUAN,20304223899
6,FATTORE,JUAN,30623008858
7,SPAVENTO,RICARDO-PONCE ALBINA,No encontrado
8,SOLIS,EDUARDO DANIEL,20137401488
9,SOLIS,EDUARDO DANIEL,20214128323
